In [28]:
from stesml.model_tools import train_model
from stesml.model_tools import test_model
from stesml.model_tools import get_T_from_h_results

from stesml.plot_tools import plot_results
from stesml.plot_tools import save_figures
from stesml.plot_tools import plot_average_error

from stesml.stes_model import stes_model

In [29]:
plot = False

In [30]:
# Model names
# NN model truncated at t <= 360: 'NN_20220708-22'
# NN model for full run time: 'NN_20220708-23'
# XGBoost model for full run time 'XGBoost_20220711-1554'
# XGBoost model truncated at t >= 360 'XGBoost_20220709-0914'

In [31]:
# Train the XGBoost model for t >= 360

data_dir = "../data/Sulfur_Models/heating/heating_all"
model_name = 'XGBoost_h_model'
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
scale = False # Normalize data to a mean of 0 and STD of 1
t_min = -1
truncated = False

In [32]:
train_model_ = False
save_model_ = False
load_model_ = True

In [33]:
if train_model_:
    parameters = stes_model.get_parameters(model_type, truncated)
    model, addendum = train_model(data_dir, model_type, target, scale, parameters, t_min=t_min)

In [34]:
if save_model_:
    stes_model.save_model(model, model_type, addendum)

In [35]:
if load_model_:
    model, addendum = stes_model.load_model(model_type, model_name)

In [36]:
t_min = -1 # If building final model, set t_min to -1 here
h_results = test_model(model, model_type, data_dir, target, scale, addendum, t_min=t_min)

RMSE: 2.6610673, R2: 0.9989617


In [37]:
h_df = h_results['test_df']

In [38]:
if plot:
    (h_df['h'] - h_df['h_hat']).hist(bins=20, range=(-30, 30))

In [39]:
if plot:
    figures = plot_results(h_df, model_type, target='h')

In [40]:
if plot:
    figures['AverageError'] = plot_average_error(h_df, target=target)

In [41]:
if plot:
    figures_directory = save_figures(figures)